In [2]:
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import root_mean_squared_error
import matplotlib.pyplot as plt
import optuna
from vacances_scolaires_france import SchoolHolidayDates

In [3]:
holiday_dates = SchoolHolidayDates()

# Fetch holidays for Zone C for specific years
zone_c_holidays_2020 = holiday_dates.holidays_for_year_and_zone(2020, 'C')
zone_c_holidays_2021 = holiday_dates.holidays_for_year_and_zone(2021, 'C')

# Combine holidays for both years
all_zone_c_holidays = list(zone_c_holidays_2020.keys()) + list(zone_c_holidays_2021.keys())

# Convert to pandas datetime
all_zone_c_holidays = pd.to_datetime(all_zone_c_holidays)

In [4]:
def encode_dates(X):
    """
    Encode date information from the 'date' column.
    Adds year, month, day, weekday, hour, holiday, and weekend indicators.
    """
    lockdown_periods = [
        ("2020-03-17", "2020-05-11"),
        ("2020-10-30", "2020-12-14"),
        ("2021-04-03", "2021-06-30"),
    ]
    
    lockdown_ranges = [
        (pd.to_datetime(start), pd.to_datetime(end)) for start, end in lockdown_periods
    ]
    
    X = X.copy()
    X["year"] = X["date_x"].dt.year
    X["month"] = X["date_x"].dt.month
    X["day"] = X["date_x"].dt.day
    X["weekday"] = X["date_x"].dt.weekday
    X["hour"] = X["date_x"].dt.hour
    X["holiday"] = X["date_x"].isin(all_zone_c_holidays).astype(int)
    X["weekend"] = (X["date_x"].dt.dayofweek > 4).astype(int)
    X["lockdown"] = X["date_x"].apply(
        lambda d: any(start <= d <= end for start, end in lockdown_ranges)
    ).astype(int)
    X['is_peak'] = X['hour'].apply(lambda x: 1 if (6 <= x < 9 or 16 <= x < 19) else 0)

    X['sin_hour'] = np.sin(2 * np.pi * X['hour'] / 24)
    X['cos_hour'] = np.cos(2 * np.pi * X['hour'] / 24)
    
    return X.drop(columns=['date_x', 'hour'])

In [5]:
def engineer_weather_features(data):
    # 1. Categorical Buckets
    data['rain_category'] = pd.cut(
        data['rr1'], bins=[-1, 0, 2, 10, float('inf')],
        labels=['No Rain', 'Light Rain', 'Moderate Rain', 'Heavy Rain']
    )
    
    data['snow_category'] = pd.cut(
        data['ht_neige'], bins=[-1, 0, 0.01, 0.05, float('inf')],
        labels=['No Snow', 'Light Snow', 'Moderate Snow', 'Heavy Snow']
    )
    
    data['is_hot_day'] = (data['t'] > 300).astype(int)  # Assuming temperature in Kelvin
    data['is_cold_day'] = (data['t'] < 283).astype(int)
    data['high_wind'] = (data['ff'] > 5).astype(int)
    
    # 3. Interaction Features
    data['rain_with_wind'] = data['rr1'] * data['ff']
    data['rolling_rain'] = data['rr1'].rolling(window=3, min_periods=1).sum()
    
    return data

In [6]:
data = pd.read_parquet(Path("data") / "train.parquet")

important_columns = ["date", "pres", "ff", "t", "u", "vv", "n", "ht_neige", "rr1"]
weather_data = pd.read_csv("./external_data/external_data.csv", usecols=important_columns)

In [7]:
weather_data["date"] = pd.to_datetime(weather_data["date"])
weather_data = weather_data.dropna(axis=1, how="all")
weather_data.set_index("date", inplace=True)
weather_data = weather_data[~weather_data.index.duplicated(keep="first")]
weather_data_interpolated = weather_data.resample("h").interpolate(method="linear")

In [8]:
covid_data_path = Path("./synthese-fra (1).csv")
columns = ["date", "nouveaux_patients_hospitalises"]
covid_data = pd.read_csv(covid_data_path, usecols=columns, parse_dates=["date"])
covid_data["date"] = pd.to_datetime(covid_data["date"]).astype("datetime64[ns]")
covid_data["date_only"] = covid_data["date"].dt.date

In [9]:
merged_data = data.merge(weather_data_interpolated, on="date", how="left")
merged_data['date_only'] = pd.to_datetime(merged_data['date']).dt.date

merged_data = merged_data.merge(covid_data, on="date_only", how="left")

missing_values = merged_data.isnull().sum()

# Display missing values
print("Missing Values per Column:")
print(missing_values)

Missing Values per Column:
counter_id                        0
counter_name                      0
site_id                           0
site_name                         0
bike_count                        0
date_x                            0
counter_installation_date         0
coordinates                       0
counter_technical_id              0
latitude                          0
longitude                         0
log_bike_count                    0
ff                                0
t                                 0
u                                 0
vv                                0
n                                 0
pres                              0
ht_neige                          0
rr1                               0
date_only                         0
date_y                            0
nouveaux_patients_hospitalises    0
dtype: int64


In [10]:
X = merged_data[["counter_name", "site_name", "date_x", "longitude", "latitude", "ff", "t", "u", "vv", "n", "pres", "ht_neige", "rr1", "nouveaux_patients_hospitalises"]]
y = merged_data["log_bike_count"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply date encoding

X_train_encoded2 = encode_dates(X_train)
X_test_encoded2 = encode_dates(X_test)

X_train_encoded3 = engineer_weather_features(X_train_encoded2)
X_test_encoded3 = engineer_weather_features(X_test_encoded2)

# Column transformer for preprocessing
categorical_features = ["counter_name", "site_name", "rain_category", "snow_category"]
numerical_features = list(X_train_encoded3.drop(columns=categorical_features).columns)

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features),
        ("num", "passthrough", numerical_features)
    ]
)

In [11]:
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 500),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1.0, 10.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 1.0, 10.0),
    }
    
    xgb_model = XGBRegressor(**params, random_state=42)
    pipeline = Pipeline([
        ("preprocessor", preprocessor),
        ("model", xgb_model)
    ])
    pipeline.fit(X_train_encoded3, y_train)
    y_pred = pipeline.predict(X_test_encoded3)
    rmse = root_mean_squared_error(y_test, y_pred)
    return rmse

# Run Optuna optimization
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100)

# Best hyperparameters
print("Best hyperparameters:", study.best_params)

best_params = study.best_params
best_model = XGBRegressor(**best_params, random_state=42)

xgboost_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("model", best_model)
])

[I 2024-12-09 06:50:43,675] A new study created in memory with name: no-name-19bb20d2-5f3d-4567-ac53-ada0cfef14be
[I 2024-12-09 06:50:46,815] Trial 0 finished with value: 0.6237242792778609 and parameters: {'n_estimators': 205, 'max_depth': 4, 'learning_rate': 0.05580944980570565, 'subsample': 0.7785560345675102, 'colsample_bytree': 0.9023207555236837, 'reg_alpha': 2.485100467332261, 'reg_lambda': 6.63169203265426}. Best is trial 0 with value: 0.6237242792778609.
[I 2024-12-09 06:50:55,501] Trial 1 finished with value: 0.35492822802443463 and parameters: {'n_estimators': 276, 'max_depth': 10, 'learning_rate': 0.10554657188010728, 'subsample': 0.709227512988829, 'colsample_bytree': 0.7197060306323557, 'reg_alpha': 3.71513850482206, 'reg_lambda': 1.3450768857775257}. Best is trial 1 with value: 0.35492822802443463.
[I 2024-12-09 06:50:58,952] Trial 2 finished with value: 0.6057048068417251 and parameters: {'n_estimators': 274, 'max_depth': 4, 'learning_rate': 0.0532845404004256, 'subsamp

Best hyperparameters: {'n_estimators': 405, 'max_depth': 10, 'learning_rate': 0.12188099239911582, 'subsample': 0.8486050543370927, 'colsample_bytree': 0.7691595212860219, 'reg_alpha': 1.8124368905332124, 'reg_lambda': 6.0302432693215415}


In [12]:
xgboost_pipeline.fit(X_train_encoded3, y_train)
y_pred = xgboost_pipeline.predict(X_test_encoded3)
final_rmse = root_mean_squared_error(y_test, y_pred)
print(f"Final XGBoost RMSE: {final_rmse:.4f}")

Final XGBoost RMSE: 0.3437


In [18]:
df_test = pd.read_parquet("./data/final_test.parquet")
df_test_merged = df_test.merge(weather_data_interpolated, on='date', how='left')
df_test_merged['date_only'] = pd.to_datetime(df_test_merged['date']).dt.date

df_test_merged = df_test_merged.merge(covid_data, on='date_only', how='left')

df_test_merged = df_test_merged.assign(**encode_dates(df_test_merged[["date_x"]]))
df_test_merged = df_test_merged.assign(**engineer_weather_features(df_test_merged))
X_test_final = df_test_merged[[
    "counter_name", "site_name", "longitude", "latitude", "ff", "t", "u", "vv", "n", "pres", "ht_neige", "rr1",
    "rain_category", "snow_category", "is_hot_day", "is_cold_day", 
    "high_wind", "rain_with_wind", "rolling_rain", "year", "month", "day", "weekday", "sin_hour", "cos_hour", 
    "is_peak", "holiday", "weekend", "lockdown", "nouveaux_patients_hospitalises"
]]

X_test_final = preprocessor.transform(X_test_final)
final_predictions = xgboost_pipeline.named_steps['model'].predict(X_test_final)

results = pd.DataFrame({"Id": np.arange(final_predictions.shape[0]), "log_bike_count": final_predictions})
results.to_csv("submission_xgboost_optuna3.csv", index=False)